In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from seaborn import scatterplot
from seaborn import regplot
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor

In [55]:
comunidades

,Codigo,CCAA,Provincia
0,AN,Andalucía,Huelva
1,AN,Andalucía,Cadiz
2,AN,Andalucía,Sevilla
3,AN,Andalucía,Jaen
4,AN,Andalucía,Cordoba
5,AN,Andalucía,Malaga
6,AN,Andalucía,Almeria
7,AN,Andalucía,Granada
8,AR,Aragón,Huesca
9,AR,Aragón,Teruel


In [51]:
ModeloCovidPorCCAA = pd.read_csv('csvTableau/ModeloCovidPorCCAA.csv',sep=';', encoding="latin-1")
comunidades = pd.read_csv('Comunidades-Provincias.csv',sep=';', encoding="latin-1")
ProvinciaCodigo = pd.DataFrame(comunidades.groupby(by=['Codigo','CCAA'], as_index=False).sum())
ProvinciaCodigo = ProvinciaCodigo[['Codigo','CCAA']]
#ProvinciaCodigo
#Agregar CCAA_iso
#ModeloCovidPorCCAA ##26258
ModeloCovidPorCCAAFull = pd.merge(ModeloCovidPorCCAA,ProvinciaCodigo, how="inner", left_on="ccaa_iso", right_on="Codigo")
ModeloCovidPorCCAAFull = ModeloCovidPorCCAAFull[['ccaa_iso', 'fecha', 'Año', 'Mes', 'Trimestre', 'num_casos', 'Periodo',
       'fecha_int', 'Estacion', 'Curso Lectivo']]
ModeloCovidPorCCAAFull

,ccaa_iso,fecha,Año,Mes,Trimestre,num_casos,Periodo,fecha_int,Estacion,Curso Lectivo
0,AN,20/3/2020,2020,3,T1,760,202001,20032020,Primavera,1
1,AN,21/3/2020,2020,3,T1,523,202001,21032020,Primavera,1
2,AN,22/3/2020,2020,3,T1,492,202001,22032020,Primavera,1
3,AN,23/3/2020,2020,3,T1,534,202001,23032020,Primavera,1
4,AN,24/3/2020,2020,3,T1,475,202001,24032020,Primavera,1
...,...,...,...,...,...,...,...,...,...,...
26253,VC,27/12/2023,2023,12,T4,0,202304,27122023,Invierno,0
26254,VC,28/12/2023,2023,12,T4,0,202304,28122023,Invierno,0
26255,VC,29/12/2023,2023,12,T4,0,202304,29122023,Invierno,0
26256,VC,30/12/2023,2023,12,T4,0,202304,30122023,Invierno,0


In [52]:
#ModeloParo = pd.read_csv('CombinadoFinalParaCovid.csv',sep=';', encoding="latin-1")
Agrupado = pd.DataFrame(ModeloCovidPorCCAAFull.groupby(by=['Periodo','ccaa_iso'], as_index=False)['num_casos'].sum())
Agrupado

,Periodo,ccaa_iso,num_casos
0,202001,AN,5296
1,202001,AR,2129
2,202001,AS,739
3,202001,CB,808
4,202001,CE,56
...,...,...,...
299,202304,ML,160
300,202304,NC,1388
301,202304,PV,217
302,202304,RI,303


In [80]:
dataParo = pd.read_csv('ParoPorCCAASexoEdad.csv',sep=';', encoding="latin-1")
dataParo['Periodo'] = dataParo['Periodo'].str.replace('T','0')
dataParo['Periodo'] = pd.to_numeric(dataParo['Periodo'], downcast='integer')
#dataParo = pd.merge(dataParo,Agrupado, on="Periodo", how="left")
#dataParo.fillna(0, inplace=True)
#dataParo['num_casos'] = pd.to_numeric(dataParo['num_casos'], downcast='integer')
dataParo = dataParo.drop(dataParo[dataParo['Total']=='..'].index)
dataParo['Total'] = pd.to_numeric(dataParo['Total'].str.replace(',','.'))
dataParo['Comunidades y Ciudades Autónomas']=dataParo['Comunidades y Ciudades Autónomas'].str[3:]
dataParo['Comunidades y Ciudades Autónomas'] = dataParo['Comunidades y Ciudades Autónomas'].apply(lambda x: x.strip())
dataParo['Comunidades y Ciudades Autónomas'] = dataParo['Comunidades y Ciudades Autónomas'].str.replace('Balears, Illes','Illes Balears').replace('Madrid, Comunidad de','Comunidad de Madrid').replace('Murcia, Región de','Región de Murcia').replace('Navarra, Comunidad Foral de','Comunidad Foral de Navarra')
dataParoConCodigo = pd.merge(dataParo,ProvinciaCodigo,how="inner", left_on='Comunidades y Ciudades Autónomas',right_on='CCAA')


In [120]:
dataParoPredic = pd.read_csv('PredictParo2.csv',sep=';',encoding="latin-1")
dataParoPredic['Periodo'] = pd.to_numeric(dataParoPredic['Periodo'], downcast='integer')
dataParoPredicConCodigo = pd.merge(dataParoPredic,ProvinciaCodigo,how="inner", left_on='Comunidades y Ciudades Autónomas',right_on='CCAA')
dataParoPredicConCovid = dataParoPredicConCodigo.merge(Agrupado, how="left", left_on=['Periodo','Codigo'], right_on =['Periodo','ccaa_iso'])
dataParoPredicConCovid['Total Paro'] = 0.0
dataParoPredicConCovid['num_casos'] = pd.to_numeric(dataParoPredicConCovid['num_casos'], downcast="integer")
dataParoPredicConCovid = dataParoPredicConCovid[['Edad', 'Sexo', 'Comunidades y Ciudades Autónomas', 'Periodo',
         'num_casos','Total Paro']]
dataParoPredicConCovid

,Edad,Sexo,Comunidades y Ciudades Autónomas,Periodo,num_casos,Total Paro
0,De 16 a 19 años,Hombres,Andalucía,202201,141788,0.0
1,De 16 a 19 años,Hombres,Andalucía,202202,805,0.0
2,De 16 a 19 años,Hombres,Andalucía,202203,573,0.0
3,De 16 a 19 años,Hombres,Andalucía,202204,242,0.0
4,De 16 a 19 años,Hombres,Andalucía,202301,43,0.0
...,...,...,...,...,...,...
1819,55 y más años,Mujeres,"Rioja, La",202204,488,0.0
1820,55 y más años,Mujeres,"Rioja, La",202301,44,0.0
1821,55 y más años,Mujeres,"Rioja, La",202302,192,0.0
1822,55 y más años,Mujeres,"Rioja, La",202303,238,0.0


In [119]:
dataParoConCovid = dataParoConCodigo.merge(Agrupado, how="left", left_on=['Periodo','Codigo'], right_on =['Periodo','ccaa_iso'])
dataParoConCovid.fillna(0,inplace=True)
dataParoConCovid['num_casos'] = pd.to_numeric(dataParoConCovid['num_casos'], downcast="integer")
dataParoConCovid.rename(columns={"Total":"Total Paro"},inplace=True)
dataParoConCovid = dataParoConCovid[['Edad', 'Sexo', 'Comunidades y Ciudades Autónomas', 'Periodo',
         'num_casos','Total Paro']]
dataParoConCovid

,Edad,Sexo,Comunidades y Ciudades Autónomas,Periodo,num_casos,Total Paro
0,De 16 a 19 años,Hombres,Andalucía,202104,202599,15.7
1,De 16 a 19 años,Hombres,Andalucía,202103,177885,16.0
2,De 16 a 19 años,Hombres,Andalucía,202102,116712,15.4
3,De 16 a 19 años,Hombres,Andalucía,202101,239482,11.6
4,De 16 a 19 años,Hombres,Andalucía,202004,198488,15.3
...,...,...,...,...,...,...
10868,55 y más años,Mujeres,Melilla,201201,0,0.2
10869,55 y más años,Mujeres,Melilla,201104,0,0.4
10870,55 y más años,Mujeres,Melilla,201102,0,0.1
10871,55 y más años,Mujeres,Melilla,201101,0,0.1


In [103]:
dataParoDummies = pd.DataFrame()
dataParoDummies =pd.get_dummies(dataParoConCovid[['Edad','Sexo','Comunidades y Ciudades Autónomas']])
dataParoConDummies = dataParoConCovid.join(dataParoDummies,lsuffix='_left', rsuffix='_right')
dataParoConDummies.columns

Index(['Edad', 'Sexo', 'Comunidades y Ciudades Autónomas', 'Periodo',
       'num_casos', 'Total Paro', 'Edad_55 y más años', 'Edad_De 16 a 19 años',
       'Edad_De 20 a 24 años', 'Edad_De 25 a 34 años', 'Edad_De 35 a 44 años',
       'Edad_De 45 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Comunidades y Ciudades Autónomas_Andalucía',
       'Comunidades y Ciudades Autónomas_Aragón',
       'Comunidades y Ciudades Autónomas_Asturias, Principado de',
       'Comunidades y Ciudades Autónomas_Canarias',
       'Comunidades y Ciudades Autónomas_Cantabria',
       'Comunidades y Ciudades Autónomas_Castilla - La Mancha',
       'Comunidades y Ciudades Autónomas_Castilla y León',
       'Comunidades y Ciudades Autónomas_Cataluña',
       'Comunidades y Ciudades Autónomas_Ceuta',
       'Comunidades y Ciudades Autónomas_Comunidad Foral de Navarra',
       'Comunidades y Ciudades Autónomas_Comunidad de Madrid',
       'Comunidades y Ciudades Autónomas_Comunitat Valenciana',
       'Comu

In [121]:
dataParoPredicDummies = pd.DataFrame()
dataParoPredicDummies =pd.get_dummies(dataParoPredicConCovid[['Edad','Sexo','Comunidades y Ciudades Autónomas']])
dataParoPredicConDummies = dataParoPredicConCovid.join(dataParoPredicDummies,lsuffix='_left', rsuffix='_right')
dataParoPredicConDummies.columns

Index(['Edad', 'Sexo', 'Comunidades y Ciudades Autónomas', 'Periodo',
       'num_casos', 'Total Paro', 'Edad_55 y más años', 'Edad_De 16 a 19 años',
       'Edad_De 20 a 24 años', 'Edad_De 25 a 34 años', 'Edad_De 35 a 44 años',
       'Edad_De 45 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Comunidades y Ciudades Autónomas_Andalucía',
       'Comunidades y Ciudades Autónomas_Aragón',
       'Comunidades y Ciudades Autónomas_Asturias, Principado de',
       'Comunidades y Ciudades Autónomas_Canarias',
       'Comunidades y Ciudades Autónomas_Cantabria',
       'Comunidades y Ciudades Autónomas_Castilla - La Mancha',
       'Comunidades y Ciudades Autónomas_Castilla y León',
       'Comunidades y Ciudades Autónomas_Cataluña',
       'Comunidades y Ciudades Autónomas_Ceuta',
       'Comunidades y Ciudades Autónomas_Comunidad Foral de Navarra',
       'Comunidades y Ciudades Autónomas_Comunidad de Madrid',
       'Comunidades y Ciudades Autónomas_Comunitat Valenciana',
       'Comu

In [123]:
X = dataParoConDummies[[ 'Periodo',
       'num_casos', 'Edad_55 y más años', 'Edad_De 16 a 19 años',
       'Edad_De 20 a 24 años', 'Edad_De 25 a 34 años', 'Edad_De 35 a 44 años',
       'Edad_De 45 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Comunidades y Ciudades Autónomas_Andalucía',
       'Comunidades y Ciudades Autónomas_Aragón',
       'Comunidades y Ciudades Autónomas_Asturias, Principado de',
       'Comunidades y Ciudades Autónomas_Canarias',
       'Comunidades y Ciudades Autónomas_Cantabria',
       'Comunidades y Ciudades Autónomas_Castilla - La Mancha',
       'Comunidades y Ciudades Autónomas_Castilla y León',
       'Comunidades y Ciudades Autónomas_Cataluña',
       'Comunidades y Ciudades Autónomas_Ceuta',
       'Comunidades y Ciudades Autónomas_Comunidad Foral de Navarra',
       'Comunidades y Ciudades Autónomas_Comunidad de Madrid',
       'Comunidades y Ciudades Autónomas_Comunitat Valenciana',
       'Comunidades y Ciudades Autónomas_Extremadura',
       'Comunidades y Ciudades Autónomas_Galicia',
       'Comunidades y Ciudades Autónomas_Illes Balears',
       'Comunidades y Ciudades Autónomas_Melilla',
       'Comunidades y Ciudades Autónomas_País Vasco',
       'Comunidades y Ciudades Autónomas_Región de Murcia',
       'Comunidades y Ciudades Autónomas_Rioja, La']]
y = dataParoConDummies[['Total Paro']]

X_predict = dataParoPredicConDummies[[ 'Periodo',
       'num_casos', 'Edad_55 y más años', 'Edad_De 16 a 19 años',
       'Edad_De 20 a 24 años', 'Edad_De 25 a 34 años', 'Edad_De 35 a 44 años',
       'Edad_De 45 a 54 años', 'Sexo_Hombres', 'Sexo_Mujeres',
       'Comunidades y Ciudades Autónomas_Andalucía',
       'Comunidades y Ciudades Autónomas_Aragón',
       'Comunidades y Ciudades Autónomas_Asturias, Principado de',
       'Comunidades y Ciudades Autónomas_Canarias',
       'Comunidades y Ciudades Autónomas_Cantabria',
       'Comunidades y Ciudades Autónomas_Castilla - La Mancha',
       'Comunidades y Ciudades Autónomas_Castilla y León',
       'Comunidades y Ciudades Autónomas_Cataluña',
       'Comunidades y Ciudades Autónomas_Ceuta',
       'Comunidades y Ciudades Autónomas_Comunidad Foral de Navarra',
       'Comunidades y Ciudades Autónomas_Comunidad de Madrid',
       'Comunidades y Ciudades Autónomas_Comunitat Valenciana',
       'Comunidades y Ciudades Autónomas_Extremadura',
       'Comunidades y Ciudades Autónomas_Galicia',
       'Comunidades y Ciudades Autónomas_Illes Balears',
       'Comunidades y Ciudades Autónomas_Melilla',
       'Comunidades y Ciudades Autónomas_País Vasco',
       'Comunidades y Ciudades Autónomas_Región de Murcia',
       'Comunidades y Ciudades Autónomas_Rioja, La']]
y_predict = dataParoPredicConDummies[['Total Paro']]

In [124]:
 BoostModelFinal = RandomForestRegressor(n_estimators = 1000, max_depth=9, random_state=0)
 BoostModelFinal.fit(X,y.values.ravel())

RandomForestRegressor(max_depth=9, n_estimators=1000, random_state=0)

In [125]:
predictions = BoostModelFinal.predict(X_predict)

In [126]:
dataParoPredicConCovid['Total Paro'] = predictions
#dataParoConCovid.to_csv('test.csv')

In [130]:
dataParoPredicConCovid.groupby('Periodo', as_index=False)['Total Paro'].sum()

,Periodo,Total Paro
0,202201,3743.771448
1,202202,3739.411991
2,202203,3739.411991
3,202204,3739.504638
4,202301,3739.425529
5,202302,3739.425529
6,202303,3739.425529
7,202304,3739.825110


In [129]:
ParoFinalPorCCAA = pd.concat([dataParoConCovid,dataParoPredicConCovid])
ParoFinalPorCCAA.to_csv('ParoFinalPorCCAAIntegrado.csv')